# Parameter definition
Choose all the settings for the database creation. Input, output dir...

In [1]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=[ 'lfw'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender','age','emotion','race']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

[20:03:20] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[20:03:20] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.8/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[20:03:20] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:03:20] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[20:03:20] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:03:20] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[20:03:20] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:03:20] ../src/nnvm/lega

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2023-01-18 20:03:21.160934: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'colab_main_dataset',
 'input_dir': '/home/jmacarulla/video_resources/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/video_resources/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7fdec8170640>,
 'databases': [<sql_face.databases.LFW at 0x7fdec8170670>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
import pandas as pd
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage

session = database.session

""" pd.read_sql_query(
    sql = session.query(Students.first_name,
                        Students.last_name).filter(
      Students.score > 80).statement, con = engine) """

# generamos un datafame con la información que necesitamos para los registros sin quality
# hacemos una query para actualizar los registros que no tienen quality


df_a = pd.read_sql_query(session.query(QualityImage,FaceImage.embeddingModel_id,FaceImage.croppedImage_id)
    .join(FaceImage)
    .filter(QualityImage.quality != None).statement, con=session.bind)


sql_b = (session.query(QualityImage, FaceImage)
    .join(FaceImage)
    .filter(QualityImage.quality == None).all())

for Q,F in sql_b:
    #recorremos el resultado y actualizamos el objeto QualityImage


    qm = Q.qualityModel_id
    em = F.embeddingModel_id
    

    


len(a)
len(b)
a[0].qualityImage_id
b[0].qualityImage_id
# for row in a:
#     print(row.first.image_id,row.second.image_id,row.get_n_common_attributes())

AttributeError: 'str' object has no attribute 'qualityModel'

In [ ]:
serfiq = None

In [ ]:
database.create_tables(serfiq=serfiq)

In [ ]:
from sql_face.tables import Image
images = database.session.query(Image).all()
len(images)

In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)